# PDR Assessor Data Analysis

In [1]:
#SF Open Data included very incomplete building SF data, so this cleans and analyzes City/County Assessor property tax data

In [1]:
import pandas as pd
import requests
import geopandas as gpd
import matplotlib.pyplot as plt
from pandas import ExcelWriter
from pandas import ExcelFile

In [2]:
pip install xlrd

You are using pip version 9.0.3, however version 20.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
#Accessing 2017-2018 property tax data
url = 'https://www.sfassessor.org/sites/default/files/uploaded/2019.8.12__SF_ASR_Secured_Roll_Data_2017-2018_0.xlsx'

import requests  
r = requests.get(url)  
open('AssessorPropertyData.xls', 'wb').write(r.content)  

32639786

In [4]:
sheet = pd.read_excel('AssessorPropertyData.xls')

In [6]:
pd.set_option("display.max.columns", None)
sheet.head()

,PROPLOC,RP1NBRCDE,RP1PRCLID,RP1VOLUME,RP1CLACDE,YRBLT,BATHS,BEDS,ROOMS,STOREYNO,UNITS,ZONE,CONSTTYPE,DEPTH,FRONT,SQFT,FBA,LAREA,LOTCODE,REPRISDATE,RP1TRACDE,OWNRPRCNT,EXEMPTYPE,RP1STACDE,RP1EXMVL2,RP1EXMVL1,ROLLYEAR,RECURRSALD,RP1FXTVAL,RP1IMPVAL,RP1LNDVAL,RP1PPTVAL
0,0000 0000 0000,08H,0001 001,1,G,1900,0,0,0,0,0,P,NaN,0.0,0.0,0,0,27965.0,NaN,NaN,1000.0,1.0,NaN,N,0,0,17,0,0,0,0,0
1,0000 0000 0000,08H,0002 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,8359.0,NaN,NaN,1000.0,1.0,NaN,N,0,0,17,0,0,0,0,0
2,0000 0000 0000,08H,0004 002,1,VCI,0,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,0.0,NaN,NaN,1000.0,1.0,NaN,N,0,0,17,0,0,0,0,0
3,0000 0286 JEFFERSON ST0000,08H,0005 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,68153.0,NaN,NaN,1000.0,1.0,NaN,N,0,0,17,0,0,0,0,0
4,0000 0000 0000,08H,0006 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,108015.0,NaN,NaN,1000.0,1.0,NaN,N,0,0,17,0,0,0,0,0


In [7]:
#Looking at the data

sheet.sample(n=10)

,PROPLOC,RP1NBRCDE,RP1PRCLID,RP1VOLUME,RP1CLACDE,YRBLT,BATHS,BEDS,ROOMS,STOREYNO,UNITS,ZONE,CONSTTYPE,DEPTH,FRONT,SQFT,FBA,LAREA,LOTCODE,REPRISDATE,RP1TRACDE,OWNRPRCNT,EXEMPTYPE,RP1STACDE,RP1EXMVL2,RP1EXMVL1,ROLLYEAR,RECURRSALD,RP1FXTVAL,RP1IMPVAL,RP1LNDVAL,RP1PPTVAL
84690,0000 2038 42ND AV0000,02B,2161 023,16,D,1947,1,0,5,1,1,RH1,D,0.0,0.0,1162,0,2996.0,NaN,NaN,1000.0,0.50000,NaN,NaN,0,0,17,0,0,31614,28020,0
50927,0000 0072 CARMEL ST0000,05E,1291 031,9,F,1912,0,0,0,0,0,NaN,NaN,0.0,0.0,1444,0,0.0,NaN,NaN,1000.0,1.00000,11,NaN,0,7000,17,31212,0,377804,566709,0
15176,0000 0336 MASON ST0000,08I,0315 018,3,O,1968,2,0,5,2,0,C3G,B,0.0,0.0,4374,0,2186.0,O,NaN,1000.0,1.00000,NaN,NaN,0,0,17,150716,0,1938000,3264000,0
83900,0000 2061 16TH AV0000,02A,2136 017,16,D,1986,3,5,10,1,1,RH1D,D,20.0,0.0,3349,1189,4670.4,R,NaN,1000.0,1.00000,11,NaN,0,7000,17,0,0,454705,214472,0
160220,0000 0055 GATES ST0000,09G,5626 026,33,D,1947,1,2,6,1,1,RH1,D,0.0,0.0,1373,130,1746.0,NaN,NaN,1000.0,1.00000,NaN,NaN,0,0,17,160822,0,637500,1487500,0
170851,0000 0448 LONDON ST0000,10C,6084 013,36,D,1917,1,0,5,1,1,RH2,D,0.0,0.0,1085,0,2495.0,NaN,NaN,1000.0,1.00000,11,NaN,0,7000,17,0,0,31031,31031,0
91904,0000 2562 15TH AV0000,04T,2415 022,18,D,1952,1,0,5,1,1,RH1,D,0.0,0.0,1303,0,3184.0,NaN,NaN,1000.0,1.00000,11,NaN,0,7000,17,0,0,150279,77019,0
198783,0043 0041 RANDOLPH ST0000,03G,7115 043,43,F,1967,4,0,14,0,2,RH2,D,0.0,0.0,3200,0,1973.0,NaN,NaN,1000.0,1.00000,NaN,NaN,0,0,17,920311,0,195371,217086,0
136372,0000 0175 BLUXOME ST0317,09D,3785 115,25,Z,1998,2,0,0,1,1,NaN,D,0.0,0.0,1027,0,0.0,NaN,NaN,1000.0,0.66666,NaN,NaN,0,0,17,91223,0,222414,361427,0
173243,0000 1375 GOETTINGEN ST0000,10E,6198 018,37,D,1952,2,3,7,1,1,RH1,D,0.0,0.0,1283,331,1873.0,NaN,NaN,1000.0,1.00000,11,NaN,0,7000,17,0,0,98642,37799,0


In [ ]:
#looking at average land value

In [5]:
sheet.RP1LNDVAL.describe()

count    2.080410e+05
mean     5.400424e+05
std      3.242311e+06
min      0.000000e+00
25%      8.076200e+04
50%      2.748560e+05
75%      5.276410e+05
max      5.835724e+08
Name: RP1LNDVAL, dtype: float64

In [6]:
sheet.RP1LNDVAL.mean()

540042.3986714158

In [8]:
#Figuring out how to merge RP1PRCLID, the parcel number, into a single 7 digit identifier
sheet.RP1PRCLID.describe()

count       208041
unique      208041
top       6694 022
freq             1
Name: RP1PRCLID, dtype: object

In [9]:
sheet['parcelno'] = sheet['RP1PRCLID']

In [10]:
sheet.head()

,PROPLOC,RP1NBRCDE,RP1PRCLID,RP1VOLUME,RP1CLACDE,YRBLT,BATHS,BEDS,ROOMS,STOREYNO,UNITS,ZONE,CONSTTYPE,DEPTH,FRONT,SQFT,FBA,LAREA,LOTCODE,REPRISDATE,RP1TRACDE,OWNRPRCNT,EXEMPTYPE,RP1STACDE,RP1EXMVL2,RP1EXMVL1,ROLLYEAR,RECURRSALD,RP1FXTVAL,RP1IMPVAL,RP1LNDVAL,RP1PPTVAL,parcelno
0,0000 0000 0000,08H,0001 001,1,G,1900,0,0,0,0,0,P,NaN,0.0,0.0,0,0,27965.0,NaN,NaN,1000.0,1.0,NaN,N,0,0,17,0,0,0,0,0,0001 001
1,0000 0000 0000,08H,0002 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,8359.0,NaN,NaN,1000.0,1.0,NaN,N,0,0,17,0,0,0,0,0,0002 001
2,0000 0000 0000,08H,0004 002,1,VCI,0,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,0.0,NaN,NaN,1000.0,1.0,NaN,N,0,0,17,0,0,0,0,0,0004 002
3,0000 0286 JEFFERSON ST0000,08H,0005 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,68153.0,NaN,NaN,1000.0,1.0,NaN,N,0,0,17,0,0,0,0,0,0005 001
4,0000 0000 0000,08H,0006 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,108015.0,NaN,NaN,1000.0,1.0,NaN,N,0,0,17,0,0,0,0,0,0006 001


In [11]:
sheet[['Block','Lot']] = sheet.parcelno.str.split(" ",expand=True,)
sheet

,PROPLOC,RP1NBRCDE,RP1PRCLID,RP1VOLUME,RP1CLACDE,YRBLT,BATHS,BEDS,ROOMS,STOREYNO,UNITS,ZONE,CONSTTYPE,DEPTH,FRONT,SQFT,FBA,LAREA,LOTCODE,REPRISDATE,RP1TRACDE,OWNRPRCNT,EXEMPTYPE,RP1STACDE,RP1EXMVL2,RP1EXMVL1,ROLLYEAR,RECURRSALD,RP1FXTVAL,RP1IMPVAL,RP1LNDVAL,RP1PPTVAL,parcelno,Block,Lot
0,0000 0000 0000,08H,0001 001,1,G,1900,0,0,0,0,0,P,NaN,0.0,0.0,0,0,27965.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0001 001,0001,001
1,0000 0000 0000,08H,0002 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,8359.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0002 001,0002,001
2,0000 0000 0000,08H,0004 002,1,VCI,0,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,0.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0004 002,0004,002
3,0000 0286 JEFFERSON ST0000,08H,0005 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,68153.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0005 001,0005,001
4,0000 0000 0000,08H,0006 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,108015.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0006 001,0006,001
5,0000 0000 0000,08H,0007 001,1,G,1900,0,0,0,0,0,C2,NaN,75.0,0.0,0,0,113437.50,R,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0007 001,0007,001
6,0000 2801 LEAVENWORTH ST0000,08H,0010 001,1,C1,1907,0,0,0,3,0,C2,C,0.0,0.0,133675,0,66124.00,NaN,NaN,1000.0,1.0000,NaN,NaN,0,0,17,110523,0,9087359,13631039,0,0010 001,0010,001
7,0495 0455 JEFFERSON ST0000,08H,0010 002,1,H,1907,0,0,252,4,0,C2,NaN,0.0,0.0,203840,0,47131.00,NaN,NaN,1000.0,1.0000,NaN,NaN,0,0,17,0,0,0,1,0,0010 002,0010,002
8,0000 2800 LEAVENWORTH ST0000,08H,0011 007,1,HC,0,0,0,0,0,0,C2,NaN,75.0,0.0,0,0,55687.50,R,NaN,1000.0,1.0000,NaN,NaN,0,0,17,41001,947696,32668516,32668516,67032,0011 007,0011,007
9,0000 0500 BEACH ST0000,08H,0011 008,1,C1,1974,7,0,36,4,29,C2,A,75.0,0.0,198525,0,57750.00,R,NaN,1000.0,1.0000,NaN,NaN,0,0,17,41001,0,16694570,32668516,0,0011 008,0011,008


In [12]:
sheet["parcelno2"] = sheet["Block"] + sheet["Lot"]

In [13]:
sheet

,PROPLOC,RP1NBRCDE,RP1PRCLID,RP1VOLUME,RP1CLACDE,YRBLT,BATHS,BEDS,ROOMS,STOREYNO,UNITS,ZONE,CONSTTYPE,DEPTH,FRONT,SQFT,FBA,LAREA,LOTCODE,REPRISDATE,RP1TRACDE,OWNRPRCNT,EXEMPTYPE,RP1STACDE,RP1EXMVL2,RP1EXMVL1,ROLLYEAR,RECURRSALD,RP1FXTVAL,RP1IMPVAL,RP1LNDVAL,RP1PPTVAL,parcelno,Block,Lot,parcelno2
0,0000 0000 0000,08H,0001 001,1,G,1900,0,0,0,0,0,P,NaN,0.0,0.0,0,0,27965.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0001 001,0001,001,0001001
1,0000 0000 0000,08H,0002 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,8359.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0002 001,0002,001,0002001
2,0000 0000 0000,08H,0004 002,1,VCI,0,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,0.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0004 002,0004,002,0004002
3,0000 0286 JEFFERSON ST0000,08H,0005 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,68153.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0005 001,0005,001,0005001
4,0000 0000 0000,08H,0006 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,108015.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0006 001,0006,001,0006001
5,0000 0000 0000,08H,0007 001,1,G,1900,0,0,0,0,0,C2,NaN,75.0,0.0,0,0,113437.50,R,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0007 001,0007,001,0007001
6,0000 2801 LEAVENWORTH ST0000,08H,0010 001,1,C1,1907,0,0,0,3,0,C2,C,0.0,0.0,133675,0,66124.00,NaN,NaN,1000.0,1.0000,NaN,NaN,0,0,17,110523,0,9087359,13631039,0,0010 001,0010,001,0010001
7,0495 0455 JEFFERSON ST0000,08H,0010 002,1,H,1907,0,0,252,4,0,C2,NaN,0.0,0.0,203840,0,47131.00,NaN,NaN,1000.0,1.0000,NaN,NaN,0,0,17,0,0,0,1,0,0010 002,0010,002,0010002
8,0000 2800 LEAVENWORTH ST0000,08H,0011 007,1,HC,0,0,0,0,0,0,C2,NaN,75.0,0.0,0,0,55687.50,R,NaN,1000.0,1.0000,NaN,NaN,0,0,17,41001,947696,32668516,32668516,67032,0011 007,0011,007,0011007
9,0000 0500 BEACH ST0000,08H,0011 008,1,C1,1974,7,0,36,4,29,C2,A,75.0,0.0,198525,0,57750.00,R,NaN,1000.0,1.0000,NaN,NaN,0,0,17,41001,0,16694570,32668516,0,0011 008,0011,008,0011008


In [14]:
#Cleaning up column names

sheet['zoning'] = sheet['ZONE']
sheet['buildingsf'] = sheet['SQFT']
sheet['landarea'] = sheet['LAREA']
sheet['landvalue'] = sheet['RP1LNDVAL']

In [15]:
sheet

,PROPLOC,RP1NBRCDE,RP1PRCLID,RP1VOLUME,RP1CLACDE,YRBLT,BATHS,BEDS,ROOMS,STOREYNO,UNITS,ZONE,CONSTTYPE,DEPTH,FRONT,SQFT,FBA,LAREA,LOTCODE,REPRISDATE,RP1TRACDE,OWNRPRCNT,EXEMPTYPE,RP1STACDE,RP1EXMVL2,RP1EXMVL1,ROLLYEAR,RECURRSALD,RP1FXTVAL,RP1IMPVAL,RP1LNDVAL,RP1PPTVAL,parcelno,Block,Lot,parcelno2,zoning,buildingsf,landarea,landvalue
0,0000 0000 0000,08H,0001 001,1,G,1900,0,0,0,0,0,P,NaN,0.0,0.0,0,0,27965.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0001 001,0001,001,0001001,P,0,27965.00,0
1,0000 0000 0000,08H,0002 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,8359.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0002 001,0002,001,0002001,C2,0,8359.00,0
2,0000 0000 0000,08H,0004 002,1,VCI,0,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,0.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0004 002,0004,002,0004002,C2,0,0.00,0
3,0000 0286 JEFFERSON ST0000,08H,0005 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,68153.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0005 001,0005,001,0005001,C2,0,68153.00,0
4,0000 0000 0000,08H,0006 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,108015.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0006 001,0006,001,0006001,C2,0,108015.00,0
5,0000 0000 0000,08H,0007 001,1,G,1900,0,0,0,0,0,C2,NaN,75.0,0.0,0,0,113437.50,R,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0007 001,0007,001,0007001,C2,0,113437.50,0
6,0000 2801 LEAVENWORTH ST0000,08H,0010 001,1,C1,1907,0,0,0,3,0,C2,C,0.0,0.0,133675,0,66124.00,NaN,NaN,1000.0,1.0000,NaN,NaN,0,0,17,110523,0,9087359,13631039,0,0010 001,0010,001,0010001,C2,133675,66124.00,13631039
7,0495 0455 JEFFERSON ST0000,08H,0010 002,1,H,1907,0,0,252,4,0,C2,NaN,0.0,0.0,203840,0,47131.00,NaN,NaN,1000.0,1.0000,NaN,NaN,0,0,17,0,0,0,1,0,0010 002,0010,002,0010002,C2,203840,47131.00,1
8,0000 2800 LEAVENWORTH ST0000,08H,0011 007,1,HC,0,0,0,0,0,0,C2,NaN,75.0,0.0,0,0,55687.50,R,NaN,1000.0,1.0000,NaN,NaN,0,0,17,41001,947696,32668516,32668516,67032,0011 007,0011,007,0011007,C2,0,55687.50,32668516
9,0000 0500 BEACH ST0000,08H,0011 008,1,C1,1974,7,0,36,4,29,C2,A,75.0,0.0,198525,0,57750.00,R,NaN,1000.0,1.0000,NaN,NaN,0,0,17,41001,0,16694570,32668516,0,0011 008,0011,008,0011008,C2,198525,57750.00,32668516


In [16]:
#creating a new and clean data frame to work with
reldata = sheet[['parcelno2', 'zoning', 'buildingsf', 'landarea', 'landvalue']].copy()

In [17]:
reldata

,parcelno2,zoning,buildingsf,landarea,landvalue
0,0001001,P,0,27965.00,0
1,0002001,C2,0,8359.00,0
2,0004002,C2,0,0.00,0
3,0005001,C2,0,68153.00,0
4,0006001,C2,0,108015.00,0
5,0007001,C2,0,113437.50,0
6,0010001,C2,133675,66124.00,13631039
7,0010002,C2,203840,47131.00,1
8,0011007,C2,0,55687.50,32668516
9,0011008,C2,198525,57750.00,32668516


In [18]:
reldata.to_csv(r'AssessorDataNew.csv')

In [19]:
#bringing existing parcel data set to merge assessor data with

In [20]:
parcelzoning = pd.read_csv('parcelwzoning.csv', low_memory=False)

In [21]:
parcelzoning.sample(n=8)

,Unnamed: 0,objectid,blklot,bldgsf,totuses,landuse,pdr,parcelarea,zoning_code,active,geometry,zoning
9111,9111,9082,0492026,9852,1678,MIXRES,0,3645.577560,RH-3,True,MULTIPOLYGON (((-122.4346706431073 37.80076807...,RH-3
153882,153882,153883,4281A020,910,0,RESIDENT,0,2479.175970,RH-2,True,MULTIPOLYGON (((-122.4017779526449 37.75142015...,RH-2
1160,1160,1160,0092021,2310,0,RESIDENT,0,1186.691217,RM-1,True,MULTIPOLYGON (((-122.41539137364461 37.8008808...,RM-1
47672,47672,47643,1899037,1050,0,RESIDENT,0,2975.841806,RH-1,True,MULTIPOLYGON (((-122.50400288516278 37.7559650...,RH-1
73947,73947,73927,3148007,0,0,VACANT,0,2479.218124,NCD,True,MULTIPOLYGON (((-122.43520022729412 37.7249050...,NCD
100561,100561,100544,6529017,2832,0,RESIDENT,0,3223.689734,RTO-M,True,MULTIPOLYGON (((-122.41882254076701 37.7499073...,RTO-M
9724,9724,9693,0679037,0,0,VACANT,0,4100.387708,RH-3,True,MULTIPOLYGON (((-122.43615881584036 37.7861967...,RH-3
58053,58053,58029,2439024,1383,0,RESIDENT,0,2942.247101,RH-1,True,MULTIPOLYGON (((-122.49637022573303 37.7390554...,RH-1


In [22]:
parcelzoning.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155468 entries, 0 to 155467
Data columns (total 12 columns):
Unnamed: 0     155468 non-null int64
objectid       155468 non-null int64
blklot         155468 non-null object
bldgsf         155468 non-null int64
totuses        155468 non-null int64
landuse        155468 non-null object
pdr            155468 non-null int64
parcelarea     155468 non-null float64
zoning_code    154673 non-null object
active         155451 non-null object
geometry       155468 non-null object
zoning         154673 non-null object
dtypes: float64(1), int64(5), object(6)
memory usage: 14.2+ MB


In [23]:
#merging data, tried two different ways to see what would preserve the most non-null data

In [24]:
combined_parcel = pd.merge(left=parcelzoning, right=reldata, how='left', left_on='blklot', right_on='parcelno2')

In [25]:
combined_parcel.sample(n=8)

,Unnamed: 0,objectid,blklot,bldgsf,totuses,landuse,pdr,parcelarea,zoning_code,active,geometry,zoning_x,parcelno2,zoning_y,buildingsf,landarea,landvalue
51397,51397,51373,2190010,1036,0,RESIDENT,0,2457.514766,RH-1,True,MULTIPOLYGON (((-122.48520853615722 37.7478602...,RH-1,2190010,RH1,1036.0,3000.00,214472.0
27803,27803,27778,1338016,4223,0,RESIDENT,0,3000.434348,RH-1,True,MULTIPOLYGON (((-122.48035977922207 37.7864863...,RH-1,1338016,RH1,4223.0,3000.00,1153240.0
151977,151977,151989,6491027,2197,0,RESIDENT,0,3038.333724,RH-1,True,MULTIPOLYGON (((-122.44779153766935 37.7083835...,RH-1,6491027,RH1,2197.0,3061.87,271909.0
73412,73412,73390,3557011,5748,0,RESIDENT,0,2465.133435,RTO,True,MULTIPOLYGON (((-122.42676840489361 37.7647327...,RTO,3557011,RH3,5748.0,2539.00,38195.0
94753,94753,94742,3984016,2240,0,RESIDENT,0,2479.170991,RH-3,True,MULTIPOLYGON (((-122.39719754118521 37.7645294...,RH-3,3984016,RH3,2240.0,2500.00,140440.0
28204,28204,28179,1293009,1225,0,RESIDENT,0,3278.503742,RH-2,True,MULTIPOLYGON (((-122.44750927690022 37.7604897...,RH-2,1293009,RH2,1225.0,3293.75,263876.0
14080,14080,14051,0989009,3930,1628,MIXRES,0,3740.570037,RH-1,True,MULTIPOLYGON (((-122.45663808165189 37.7888513...,RH-1,0989009,RH1,3930.0,3763.00,1985683.0
9006,9006,8977,0670002A,2720,0,MIXRES,0,1546.339562,NCD,True,MULTIPOLYGON (((-122.42065058091144 37.7884151...,NCD,0670002A,POLK,2720.0,1560.00,700607.0


In [26]:
combined_parcel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 155468 entries, 0 to 155467
Data columns (total 17 columns):
Unnamed: 0     155468 non-null int64
objectid       155468 non-null int64
blklot         155468 non-null object
bldgsf         155468 non-null int64
totuses        155468 non-null int64
landuse        155468 non-null object
pdr            155468 non-null int64
parcelarea     155468 non-null float64
zoning_code    154673 non-null object
active         155451 non-null object
geometry       155468 non-null object
zoning_x       154673 non-null object
parcelno2      145546 non-null object
zoning_y       140671 non-null object
buildingsf     145546 non-null float64
landarea       145546 non-null float64
landvalue      145546 non-null float64
dtypes: float64(4), int64(5), object(8)
memory usage: 21.4+ MB


In [27]:
#second merged data variable

In [28]:
combined_assessor = pd.merge(left=reldata, right=parcelzoning, how='left', left_on='parcelno2', right_on='blklot')

In [29]:
combined_assessor.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 208041 entries, 0 to 208040
Data columns (total 17 columns):
parcelno2      190328 non-null object
zoning_x       172651 non-null object
buildingsf     208041 non-null int64
landarea       208041 non-null float64
landvalue      208041 non-null int64
Unnamed: 0     145546 non-null float64
objectid       145546 non-null float64
blklot         145546 non-null object
bldgsf         145546 non-null float64
totuses        145546 non-null float64
landuse        145546 non-null object
pdr            145546 non-null float64
parcelarea     145546 non-null float64
zoning_code    144769 non-null object
active         145546 non-null object
geometry       145546 non-null object
zoning_y       144769 non-null object
dtypes: float64(7), int64(2), object(8)
memory usage: 28.6+ MB


In [30]:
combined_assessor.sample(n=8)

,parcelno2,zoning_x,buildingsf,landarea,landvalue,Unnamed: 0,objectid,blklot,bldgsf,totuses,landuse,pdr,parcelarea,zoning_code,active,geometry,zoning_y
170152,6052001C,RH1,1593,2186.0,462683,117132.0,117120.0,6052001C,1593.0,0.0,RESIDENT,0.0,2168.757218,RH-1,True,MULTIPOLYGON (((-122.40602867129323 37.7252931...,RH-1
11049,NaN,RM4,756,0.0,326199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15164,0315004,C3R,23217,3149.0,390324,5802.0,5784.0,0315004,23217.0,28877.0,MIXED,0.0,3241.785688,C-3-R,True,MULTIPOLYGON (((-122.40818698612759 37.7865681...,C-3-R
91766,2412013,RH1D,1730,3600.0,240098,60064.0,60040.0,2412013,1730.0,0.0,RESIDENT,0.0,3588.051346,RH-1(D),True,MULTIPOLYGON (((-122.46918741508608 37.7419444...,RH-1(D)
82192,2090042,RH1,1476,2996.0,28020,50446.0,50422.0,2090042,1476.0,0.0,RESIDENT,0.0,2953.313557,RH-1,True,MULTIPOLYGON (((-122.50361103536814 37.7503071...,RH-1
98142,2656020,RH2,2338,2164.0,1660050,68860.0,68837.0,2656020,1375.0,0.0,RESIDENT,0.0,2262.518265,RH-2,True,MULTIPOLYGON (((-122.44263642319845 37.7611458...,RH-2
156964,NaN,M1,35173,61698.0,1212712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29690,0715023,NC3,933,0.0,341693,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
combined_assessor.loc[combined_assessor.zoning_x=='PDR2']

,parcelno2,zoning_x,buildingsf,landarea,landvalue,Unnamed: 0,objectid,blklot,bldgsf,totuses,landuse,pdr,parcelarea,zoning_code,active,geometry,zoning_y
138477,3821009,PDR2,0,13358.00,39989,92464.0,92458.0,3821009,0.0,0.0,VACANT,0.0,13381.795069,PDR-1-D,True,MULTIPOLYGON (((-122.39876573712424 37.7664213...,PDR-1-D
146248,4288004,PDR2,31546,52200.00,3802482,94166.0,94157.0,4288004,31104.0,6292.0,MIXED,3368.0,49446.269364,PDR-2,True,MULTIPOLYGON (((-122.39348424412417 37.7506985...,PDR-2
146249,4288005,PDR2,64000,81237.00,8175338,94167.0,94158.0,4288005,64000.0,19932.0,PDR,17932.0,85616.458300,PDR-2,True,MULTIPOLYGON (((-122.39354920559127 37.7500063...,PDR-2
146250,4288006,PDR2,43200,64800.00,3591233,94168.0,94159.0,4288006,43200.0,55300.0,MIXED,10914.0,63526.786658,PDR-2,True,MULTIPOLYGON (((-122.39462070874185 37.7499444...,PDR-2
146253,4290011,PDR2,9300,9361.00,838063,94327.0,94318.0,4290011,9300.0,4443.0,MIXED,0.0,9578.749982,PDR-2,True,MULTIPOLYGON (((-122.39259612029906 37.7504734...,PDR-2
146630,4343001B,PDR2,35114,69996.00,7767079,103486.0,103464.0,4343001B,35114.0,5168.0,PDR,5168.0,69618.193281,PDR-2,True,MULTIPOLYGON (((-122.397193308955 37.749182371...,PDR-2
146631,4343001D,PDR2,19232,133297.00,3728545,103487.0,103465.0,4343001D,19232.0,2246.0,RETAIL/ENT,0.0,128261.904609,PDR-2,True,MULTIPOLYGON (((-122.40166401819543 37.7470196...,PDR-2
146646,4343030,PDR2,6834,17484.00,929495,94343.0,94334.0,4343030,6834.0,0.0,PDR,0.0,19108.892097,PDR-2,True,MULTIPOLYGON (((-122.40169703643947 37.7470185...,PDR-2
146650,4346003,PDR2,74122,343882.00,34173315,94192.0,94183.0,4346003,74122.0,0.0,PDR,0.0,343542.118835,PDR-2,True,MULTIPOLYGON (((-122.39372432462824 37.7496763...,PDR-2
146653,NaN,PDR2,108793,181269.00,20415781,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
#make new zoning column as a string data type based on existing zoning_x column

In [33]:
combined_assessor['finzone'] = combined_assessor['zoning_x'].astype(str)

In [34]:
combined_assessor.head()

,parcelno2,zoning_x,buildingsf,landarea,landvalue,Unnamed: 0,objectid,blklot,bldgsf,totuses,landuse,pdr,parcelarea,zoning_code,active,geometry,zoning_y,finzone
0,0001001,P,0,27965.0,0,147396.0,147394.0,0001001,0.0,0.0,PDR,0.0,28268.231998,P,True,MULTIPOLYGON (((-122.4210628988343 37.80860124...,P,P
1,0002001,C2,0,8359.0,0,72.0,73.0,0002001,0.0,0.0,PDR,0.0,8920.300030,C-2,True,MULTIPOLYGON (((-122.41981482898801 37.8087616...,C-2,C2
2,0004002,C2,0,0.0,0,6.0,7.0,0004002,0.0,20509.0,VACANT,0.0,31183.392188,C-2,True,MULTIPOLYGON (((-122.41379081959624 37.8085666...,C-2,C2
3,0005001,C2,0,68153.0,0,7.0,8.0,0005001,0.0,43725.0,PDR,0.0,69014.270707,C-2,True,MULTIPOLYGON (((-122.41593580428093 37.8082939...,C-2,C2
4,0006001,C2,0,108015.0,0,8.0,9.0,0006001,0.0,28728.0,PDR,0.0,106916.225315,C-2,True,MULTIPOLYGON (((-122.41758109128742 37.8080868...,C-2,C2


In [35]:
#filter data based on zoning column to only include PDR districts

In [36]:
filteredPDR3 = combined_assessor[combined_assessor['finzone'].str.contains("PDR")]

In [37]:
filteredPDR3.sample(n=14)

,parcelno2,zoning_x,buildingsf,landarea,landvalue,Unnamed: 0,objectid,blklot,bldgsf,totuses,landuse,pdr,parcelarea,zoning_code,active,geometry,zoning_y,finzone
118596,3571005,PDR1G,7931,9380.0,1633779,78845.0,78827.0,3571005,7931.0,0.0,RETAIL/ENT,0.0,9323.252379,PDR-1-G,True,MULTIPOLYGON (((-122.41661724763837 37.7641329...,PDR-1-G,PDR1G
140086,4015006,PDR1G,10000,5000.0,3168798,95930.0,95924.0,4015006,10000.0,21732.0,MIXED,7732.0,4975.020115,PDR-1-G,True,MULTIPOLYGON (((-122.4085895433119 37.76276056...,PDR-1-G,PDR1G
160997,5657009D,PDR1G,18833,22000.0,2278221,126059.0,126048.0,5657009D,18833.0,2184.0,MIPS,0.0,22693.107574,PDR-1-G,True,MULTIPOLYGON (((-122.40880330105348 37.7403877...,PDR-1-G,PDR1G
145047,4227001,PDR1G,3844,3000.0,1304799,102638.0,102615.0,4227001,4000.0,4236.0,MIPS,0.0,2988.160723,PDR-1-G,True,MULTIPOLYGON (((-122.39105613842756 37.7550356...,PDR-1-G,PDR1G
148476,4786070,PDR2,2500,2500.0,344412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PDR2
159453,5596001,PDR2,5313,7923.0,528122,155456.0,155457.0,5596001,5313.0,3000.0,RETAIL/ENT,0.0,6915.819415,PDR-2,True,MULTIPOLYGON (((-122.40541982109421 37.7431508...,PDR-2,PDR2
149146,4850038,PDR2,4784,5000.0,530519,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PDR2
140127,4019003,PDR1G,6212,3110.0,939019,92598.0,92591.0,4019003,6212.0,1725.0,RETAIL/ENT,0.0,3064.066470,PDR-1-G,True,MULTIPOLYGON (((-122.41244769005871 37.7620538...,PDR-1-G,PDR1G
149614,4912001,PDR2,9075,7496.0,401372,89362.0,89363.0,4912001,9075.0,2742.0,PDR,2742.0,7409.757700,PDR-2,True,MULTIPOLYGON (((-122.3927806813064 37.72261969...,PDR-2,PDR2
156471,5415001,PDR2,16075,19335.0,2164521,89697.0,89694.0,5415001,16075.0,17000.0,PDR,17000.0,20088.840435,PDR-2,True,MULTIPOLYGON (((-122.39516215510433 37.7294693...,PDR-2,PDR2


In [38]:
filteredPDR3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 194 entries, 115261 to 161031
Data columns (total 18 columns):
parcelno2      178 non-null object
zoning_x       194 non-null object
buildingsf     194 non-null int64
landarea       194 non-null float64
landvalue      194 non-null int64
Unnamed: 0     139 non-null float64
objectid       139 non-null float64
blklot         139 non-null object
bldgsf         139 non-null float64
totuses        139 non-null float64
landuse        139 non-null object
pdr            139 non-null float64
parcelarea     139 non-null float64
zoning_code    139 non-null object
active         139 non-null object
geometry       139 non-null object
zoning_y       139 non-null object
finzone        194 non-null object
dtypes: float64(7), int64(2), object(9)
memory usage: 28.8+ KB


In [39]:
filteredPDR3.landarea.sum()

5130638.09

In [40]:
filteredPDR3.parcelarea.sum()

3979176.367905817

In [41]:
filteredPDR3.to_csv(r'filteredpdr3.csv')

In [42]:
#playing around with which zoning column is most reliable to filter off of

In [43]:
combined_parcel['finzone2'] = combined_parcel['zoning_y'].astype(str)

In [44]:
filteredPDR4 = combined_parcel[combined_parcel['finzone2'].str.contains("PDR")]

In [45]:
filteredPDR4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 139 entries, 69876 to 155456
Data columns (total 18 columns):
Unnamed: 0     139 non-null int64
objectid       139 non-null int64
blklot         139 non-null object
bldgsf         139 non-null int64
totuses        139 non-null int64
landuse        139 non-null object
pdr            139 non-null int64
parcelarea     139 non-null float64
zoning_code    139 non-null object
active         139 non-null object
geometry       139 non-null object
zoning_x       139 non-null object
parcelno2      139 non-null object
zoning_y       139 non-null object
buildingsf     139 non-null float64
landarea       139 non-null float64
landvalue      139 non-null float64
finzone2       139 non-null object
dtypes: float64(4), int64(5), object(9)
memory usage: 20.6+ KB


In [46]:
filteredpdrorig = pd.read_csv('filteredpdr.csv', low_memory=False)

In [47]:
filteredpdrorig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1448 entries, 0 to 1447
Data columns (total 12 columns):
Unnamed: 0     1448 non-null int64
objectid       1448 non-null int64
blklot         1448 non-null object
bldgsf         1448 non-null int64
totuses        1448 non-null int64
landuse        1448 non-null object
pdr            1448 non-null int64
parcelarea     1448 non-null float64
zoning_code    1448 non-null object
active         1448 non-null bool
geometry       1448 non-null object
zoning         1448 non-null object
dtypes: bool(1), float64(1), int64(5), object(5)
memory usage: 125.9+ KB


In [48]:
filteredPDR4.sample(n=10)

,Unnamed: 0,objectid,blklot,bldgsf,totuses,landuse,pdr,parcelarea,zoning_code,active,geometry,zoning_x,parcelno2,zoning_y,buildingsf,landarea,landvalue,finzone2
103486,103486,103464,4343001B,35114,5168,PDR,5168,69618.193281,PDR-2,True,MULTIPOLYGON (((-122.397193308955 37.749182371...,PDR-2,4343001B,PDR2,35114.0,69996.0,7767079.0,PDR2
89362,89362,89363,4912001,9075,2742,PDR,2742,7409.757700,PDR-2,True,MULTIPOLYGON (((-122.3927806813064 37.72261969...,PDR-2,4912001,PDR2,9075.0,7496.0,401372.0,PDR2
83214,83214,83198,3935004,87900,30601,PDR,26740,68557.396651,PDR-1-D,True,MULTIPOLYGON (((-122.404097503814 37.766237179...,PDR-1-D,3935004,PDR1D,90056.0,67602.0,27959983.0,PDR1D
85343,85343,85328,3926002,29502,0,MIXED,0,15902.647508,PDR-1-G,True,MULTIPOLYGON (((-122.41256513838024 37.7655914...,PDR-1-G,3926002,PDR1G,29502.0,18410.0,10773702.0,PDR1G
107690,107690,107672,5572031,3605,0,PDR,0,3656.960484,PDR-2,True,MULTIPOLYGON (((-122.40370943779656 37.7426369...,PDR-2,5572031,PDR2,3605.0,3485.0,517777.0,PDR2
104395,104395,104393,4812041,9200,0,PDR,0,2943.725237,PDR-2,True,MULTIPOLYGON (((-122.38549136742158 37.7255789...,PDR-2,4812041,PDR2,9200.0,2740.0,53767.0,PDR2
110173,110173,110159,5585004,17100,19733,PDR,19733,43160.696272,PDR-2,True,MULTIPOLYGON (((-122.40185190182133 37.7400715...,PDR-2,5585004,PDR2,17100.0,43769.0,792888.0,PDR2
88851,88851,88850,4910021,5000,4785,PDR,4785,4974.989312,PDR-2,True,MULTIPOLYGON (((-122.39160931059091 37.7225624...,PDR-2,4910021,PDR2,5000.0,4996.0,587274.0,PDR2
86600,86600,86586,4829007,0,1860,PDR,1860,7470.812736,PDR-1-B,False,MULTIPOLYGON (((-122.39009920389658 37.7273763...,PDR-1-B,4829007,PDR1B,0.0,7500.0,186339.0,PDR1B
90580,90580,90577,5332026,2880,0,PDR,0,3577.372650,PDR-2,True,MULTIPOLYGON (((-122.40090303925706 37.7391760...,PDR-2,5332026,PDR2,2880.0,3598.0,424415.0,PDR2


In [62]:
PDRDandG = filteredPDR4[(filteredPDR4['finzone2'].str.contains("PDR1D")) | (filteredPDR4['finzone2'].str.contains("PDR1G"))]

In [63]:
PDRDandG.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52 entries, 69876 to 154182
Data columns (total 18 columns):
Unnamed: 0     52 non-null int64
objectid       52 non-null int64
blklot         52 non-null object
bldgsf         52 non-null int64
totuses        52 non-null int64
landuse        52 non-null object
pdr            52 non-null int64
parcelarea     52 non-null float64
zoning_code    52 non-null object
active         52 non-null object
geometry       52 non-null object
zoning_x       52 non-null object
parcelno2      52 non-null object
zoning_y       52 non-null object
buildingsf     52 non-null float64
landarea       52 non-null float64
landvalue      52 non-null float64
finzone2       52 non-null object
dtypes: float64(4), int64(5), object(9)
memory usage: 7.7+ KB


In [64]:
PDRDandG.sample(n=30)

,Unnamed: 0,objectid,blklot,bldgsf,totuses,landuse,pdr,parcelarea,zoning_code,active,geometry,zoning_x,parcelno2,zoning_y,buildingsf,landarea,landvalue,finzone2
96244,96244,96237,4015003,51561,12000,PDR,12000,17388.699829,PDR-1-G,True,MULTIPOLYGON (((-122.40859064355014 37.7619402...,PDR-1-G,4015003,PDR1G,51561.0,17184.0,8772000.0,PDR1G
82996,82996,82979,3574114,7913,0,PDR,0,3879.684177,PDR-1-G,True,MULTIPOLYGON (((-122.41518349914048 37.7625565...,PDR-1-G,3574114,PDR1G,7913.0,3920.0,653663.0,PDR1G
93648,93648,93639,4294003,16600,12618,MIXED,7132,20724.932701,PDR-1-G,True,MULTIPOLYGON (((-122.38885511315166 37.7523962...,PDR-1-G,4294003,PDR1G,16600.0,20599.0,845253.0,PDR1G
141171,141171,141167,4294017,14315,0,PDR,0,12604.039557,PDR-1-G,True,MULTIPOLYGON (((-122.38914650008785 37.7517598...,PDR-1-G,4294017,PDR1G,14315.0,12501.0,1192218.0,PDR1G
102372,102372,102371,4229003,25500,4063,PDR,4063,19866.692021,PDR-1-G,True,MULTIPOLYGON (((-122.38905096918313 37.7544092...,PDR-1-G,4229003,PDR1G,25500.0,20000.0,2157703.0,PDR1G
85308,85308,85294,3926001,5043,0,MIXED,0,16845.334734,PDR-1-G,True,MULTIPOLYGON (((-122.41260022552292 37.7659612...,PDR-1-G,3926001,PDR1G,5043.0,16827.0,1901241.0,PDR1G
69876,69876,69853,3552013,0,5000,PDR,0,13212.498674,PDR-1-G,True,MULTIPOLYGON (((-122.41651291865504 37.7653523...,PDR-1-G,3552013,PDR1G,0.0,13198.0,155228.0,PDR1G
81663,81663,81647,3526019B,10000,0,PDR,0,10250.585162,PDR-1-G,True,MULTIPOLYGON (((-122.40882113950708 37.7704921...,PDR-1-G,3526019B,PDR1G,10000.0,9378.0,1801865.0,PDR1G
95929,95929,95923,4015004,24033,23300,MIPS,0,20120.978550,PDR-1-G,True,MULTIPOLYGON (((-122.40862708071916 37.7623170...,PDR-1-G,4015004,PDR1G,24033.0,19998.0,2406515.0,PDR1G
83675,83675,83659,3531027,20110,3618,RETAIL/ENT,0,12594.065629,PDR-1-G,True,MULTIPOLYGON (((-122.41922828869302 37.7687108...,PDR-1-G,3531027,PDR1G,20110.0,11904.0,3055789.0,PDR1G


In [72]:
PDRDandG["farvalue"] = (PDRDandG["buildingsf"] / PDRDandG["landarea"])
computeFAR = PDRDandG[['farvalue','geometry','finzone2','parcelno2','landvalue','buildingsf','landarea']].copy()
computeFAR.head()

/srv/app/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,farvalue,geometry,finzone2,parcelno2,landvalue,buildingsf,landarea
69876,0.000000,MULTIPOLYGON (((-122.41651291865504 37.7653523...,PDR1G,3552013,155228.0,0.0,13198.0
78845,0.845522,MULTIPOLYGON (((-122.41661724763837 37.7641329...,PDR1G,3571005,1633779.0,7931.0,9380.0
80978,2.139429,MULTIPOLYGON (((-122.41523054003258 37.7630529...,PDR1G,3574008,1448668.0,6552.0,3062.5
80979,1.956571,MULTIPOLYGON (((-122.41519180346006 37.7626441...,PDR1G,3574009,2419171.0,35952.0,18375.0
81296,1.646332,MULTIPOLYGON (((-122.41429255369673 37.7622301...,PDR1G,3573003,10562452.0,33195.0,20163.0


In [78]:
filteredFAR = computeFAR[computeFAR['farvalue'] < 0.3]
filteredFAR.landarea.sum()

144272.0

In [79]:
filteredFAR.describe()

,farvalue,landvalue,buildingsf,landarea
count,9.000000,9.000000e+00,9.000000,9.000000
mean,0.040933,2.414404e+06,636.666667,16030.222222
std,0.099661,3.574476e+06,1667.922285,11453.764739
min,0.000000,0.000000e+00,0.000000,1005.000000
25%,0.000000,1.552280e+05,0.000000,10000.000000
50%,0.000000,1.697664e+06,0.000000,14806.000000
75%,0.000000,1.901241e+06,0.000000,23642.000000
max,0.299697,1.119620e+07,5043.000000,32224.000000


In [80]:
filteredFAR.to_csv(r'filteredFAR.csv')